# Get mixscape labels

In [ ]:
# Load packages.
library(Seurat)
library(SeuratData)
library(ggplot2)
library(patchwork)
library(scales)
library(dplyr)
library(reshape2)

# Download dataset using SeuratData.
InstallData(ds = "thp1.eccite")

# Setup custom theme for plotting.
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

In [2]:
# Load object.
eccite <- LoadData(ds = "thp1.eccite")

# Normalize protein.
eccite <- NormalizeData(
  object = eccite, 
  assay = "ADT", 
  normalization.method = "CLR", 
  margin = 2)

Normalizing across cells



In [3]:
# Prepare RNA assay for dimensionality reduction: 
# Normalize data, find variable features and scale data.
DefaultAssay(object = eccite) <- 'RNA'
eccite <- NormalizeData(object = eccite) %>% FindVariableFeatures() %>% ScaleData()

# Run Principle Component Analysis (PCA) to reduce the dimensionality of the data.
eccite <- RunPCA(object = eccite)

# Run Uniform Manifold Approximation and Projection (UMAP) to visualize clustering in 2-D.
eccite <- RunUMAP(object = eccite, dims = 1:40)

20:51:06 Annoy recall = 100%

20:51:06 Commencing smooth kNN distance calibration using 1 thread

20:51:07 Initializing from normalized Laplacian + noise

20:51:08 Commencing optimization for 200 epochs, with 910832 positive edges

20:51:17 Optimization finished



In [4]:
# Calculate perturbation signature (PRTB).
eccite<- CalcPerturbSig(
  object = eccite, 
  assay = "RNA", 
  slot = "data", 
  gd.class ="gene", 
  nt.cell.class = "NT", 
  reduction = "pca", 
  ndims = 40, 
  num.neighbors = 20, 
  split.by = "replicate", 
  new.assay.name = "PRTB")

# Prepare PRTB assay for dimensionality reduction: 
# Normalize data, find variable features and center data.
DefaultAssay(object = eccite) <- 'PRTB'

# Use variable features from RNA assay.
VariableFeatures(object = eccite) <- VariableFeatures(object = eccite[["RNA"]])
eccite <- ScaleData(object = eccite, do.scale = F, do.center = T)

# Run PCA to reduce the dimensionality of the data.
eccite <- RunPCA(object = eccite, reduction.key = 'prtbpca', reduction.name = 'prtbpca')

# Run UMAP to visualize clustering in 2-D.
eccite <- RunUMAP(
  object = eccite, 
  dims = 1:40, 
  reduction = 'prtbpca', 
  reduction.key = 'prtbumap', 
  reduction.name = 'prtbumap')

20:51:47 Annoy recall = 100%

20:51:47 Commencing smooth kNN distance calibration using 1 thread

20:51:48 Initializing from normalized Laplacian + noise

20:51:48 Commencing optimization for 200 epochs, with 842656 positive edges

20:51:57 Optimization finished

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from prtbumap to prtbumap_”
Warning message:
“All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to prtbumap_”


In [8]:
eccite <- RunMixscape(
  object = eccite, 
  assay = "PRTB", 
  slot = "scale.data", 
  labels = "gene", 
  nt.class.name = "NT", 
  min.de.genes = 5, 
  iter.num = 10, 
  de.assay = "RNA", 
  verbose = F,
  prtb.type = "KO")

# Calculate percentage of KO cells for all target gene classes.
df <- prop.table(table(eccite$mixscape_class.global, eccite$NT),2)

df2 <- reshape2::melt(df)
df2$Var2 <- as.character(df2$Var2)
test <- df2[which(df2$Var1 == "KO"),]
test <- test[order(test$value, decreasing = T),]
new.levels <- test$Var2
df2$Var2 <- factor(df2$Var2, levels = new.levels )
df2$Var1 <- factor(df2$Var1, levels = c("NT", "NP", "KO"))
df2$gene <- sapply(as.character(df2$Var2), function(x) strsplit(x, split = "g")[[1]][1])
df2$guide_number <- sapply(as.character(df2$Var2), 
                           function(x) strsplit(x, split = "g")[[1]][2])
df3 <- df2[-c(which(df2$gene == "NT")),]


Warning message in FindMarkers.default(object = data.use, slot = data.slot, counts = counts, :
“No features pass logfc.threshold threshold; returning empty data.frame”


number of iterations= 95 
number of iterations= 187 
number of iterations= 172 
number of iterations= 18 
number of iterations= 6 
number of iterations= 18 
number of iterations= 11 
number of iterations= 11 
number of iterations= 59 
number of iterations= 43 
number of iterations= 42 
number of iterations= 19 
number of iterations= 12 
number of iterations= 12 
number of iterations= 23 
number of iterations= 19 
number of iterations= 19 
number of iterations= 51 
number of iterations= 51 
number of iterations= 51 
number of iterations= 36 
number of iterations= 26 
number of iterations= 25 
number of iterations= 20 
number of iterations= 12 
number of iterations= 12 
number of iterations= 17 
number of iterations= 15 
number of iterations= 14 
number of iterations= 13 
number of iterations= 73 
number of iterations= 46 
number of iterations= 41 


In [10]:
write.csv(eccite[[]], '/data_volume/scito/eccite/mixscape_meta.csv')